In [8]:
%load_ext autoreload
%autoreload 2

import requests
import sys
sys.path.append('..')
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook as tqdm
import time
import numpy as np
import pandas as pd
import nltk   
import unicodedata
from html.parser import HTMLParser
import re
from tqdm import tqdm
import glob
from utils import *
import pathlib
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import requests

agent = { 'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0',
          'Accept-Language': 'pt-BR'}

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:

#url = r"https://www.amazon.com.br/Smartphone-Xiaomi-64GB-Piano-Black/dp/B07QH823FV/ref=pd_sbs_107_5/138-4765066-9703910?_encoding=UTF8&pd_rd_i=B07QH823FV&pd_rd_r=307fed63-71d8-4c9e-aacc-82a0ddc80cd9&pd_rd_w=x0jYi&pd_rd_wg=88HJ1&pf_rd_p=80c6065d-57d3-41bf-b15e-ee01dd80424f&pf_rd_r=7N8T5AJWRZ6VPNQTB88Z&psc=1&refRID=7N8T5AJWRZ6VPNQTB88Z"


# Generic Functions

In [85]:
def levenshtein(word1, word2):
    """
    https://medium.com/@yash_agarwal2/soundex-and-levenshtein-distance-in-python-8b4b56542e9e
    https://en.wikipedia.org/wiki/Levenshtein_distance
    :param word1:
    :param word2:
    :return:
    """
    word2 = word2.lower()
    word1 = word1.lower()
    matrix = [[0 for x in range(len(word2) + 1)] for x in range(len(word1) + 1)]

    for x in range(len(word1) + 1):
        matrix[x][0] = x
    for y in range(len(word2) + 1):
        matrix[0][y] = y

    for x in range(1, len(word1) + 1):
        for y in range(1, len(word2) + 1):
            if word1[x - 1] == word2[y - 1]:
                matrix[x][y] = min(
                    matrix[x - 1][y] + 1,
                    matrix[x - 1][y - 1],
                    matrix[x][y - 1] + 1
                )
            else:
                matrix[x][y] = min(
                    matrix[x - 1][y] + 1,
                    matrix[x - 1][y - 1] + 1,
                    matrix[x][y - 1] + 1
                )

    return matrix[len(word1)][len(word2)]

def nomr_l(word1, word2):
    return levenshtein(word1, word2) / max(len(word1), len(word2))

def html_to_np(table):
    result = []
    all_rows = table.find_all('tr')
    for row in all_rows:
        result.append([])
        all_cols = row.find_all('td')
        for col in all_cols:
            col_text = [s.strip(' ') for s in col.find_all(text=True)]
            col_text  = ''.join(col_text) # it should not change anything
            result[-1].append(col_text)
    return np.array(result)


def np_to_dict(table, fields_dict):
    new_dict = fields_dict.copy()
    for cols in table:
        if len(cols) != 2:
            continue
        for key in fields_dict.keys():
            if nomr_l(key, cols[0]) <= 0.25:
                new_dict[key] = cols[1].replace("\n",'')
            #print(nomr_l(key, cols[0]), key, cols[0])
    return new_dict

## Amazom

In [86]:
fields_amazom = {'Preço': '',
        'Sistema operacional': '',
        'RAM': '',
        'Capacidade de armazenamento digital': '',
        'Modelo': '',
        'Cor': '',  
        'Tamanho da tela':'',
        'Avaliações de clientes': ''
       }

url = r"https://www.amazon.com.br/Smartphone-Xiaomi-64GB-Piano-Black/dp/B07QH823FV/ref=pd_sbs_107_5/138-4765066-9703910?_encoding=UTF8&pd_rd_i=B07QH823FV&pd_rd_r=307fed63-71d8-4c9e-aacc-82a0ddc80cd9&pd_rd_w=x0jYi&pd_rd_wg=88HJ1&pf_rd_p=80c6065d-57d3-41bf-b15e-ee01dd80424f&pf_rd_r=7N8T5AJWRZ6VPNQTB88Z&psc=1&refRID=7N8T5AJWRZ6VPNQTB88Z"

code        = requests.get(url, headers=agent)
plain       = code.text
s           = BeautifulSoup(plain, "html.parser")

In [87]:
tb = html_to_np(s)
np_to_dict(tb, fields_amazom)

{'Preço': 'R$1.998,00',
 'Sistema operacional': 'android',
 'RAM': '6',
 'Capacidade de armazenamento digital': '6 GB',
 'Modelo': 'MZB7606EU',
 'Cor': 'Preto',
 'Tamanho da tela': '5.97 polegadas',
 'Avaliações de clientes': '4.7 de 5 estrelas      109 avaliações de clientes'}

## Magazine Luiza

In [115]:
fields_luiza = {'Preço': '',
        'Sistema operacional': '',
        'Memória RAM': '',
        'Armazenamento interno': '',
        'Modelo': '',
        'Cor': '',  
        'Tamanho da tela':'',
        'Avaliações de clientes': '',
        'Marca' : ''
       }

def np_to_dict(table, fields_dict):
    new_dict = fields_dict.copy()
    for cols in table:
        if len(cols) < 2:
            continue
        for key in fields_dict.keys():
            if nomr_l(key, cols[0]) <= 0.25:
                new_dict[key] = cols[1].replace("\n",'')
            #print(nomr_l(key, cols[0]), key, cols[0])
    return new_dict

In [122]:
url = r"https://www.magazineluiza.com.br/smartphone-motorola-g7-play-32gb-indigo-4g-2gb-ram-tela-57-cam-13mp-cam-selfie-8mp/p/155549300/te/mtgp/"
code        = requests.get(url, headers=agent)
plain       = code.text
s           = BeautifulSoup(plain, "html.parser")

In [123]:
tb = html_to_np(s)
np_to_dict(tb, fields_luiza)

{'Preço': '',
 'Sistema operacional': 'Android Pie',
 'Memória RAM': '2GB',
 'Armazenamento interno': '32GB (A memória disponível para uso do consumidor pode sofrer variações, conforme sistema operacional, aplicativos e/ou outros fatores)',
 'Modelo': 'G7 Play',
 'Cor': 'Indigo',
 'Tamanho da tela': '5,7"',
 'Avaliações de clientes': '',
 'Marca': 'Motorola'}

# IByte

In [164]:
url = r"https://www.ibyte.com.br/smartphone-samsung-galaxy-j8-64gb-tela-6-octa-core-1-8ghz-4g-16mp-f1-7-5mp-f1-9-dual-cam-preto.html"
code        = requests.get(url, headers=agent)
plain       = code.text
s           = BeautifulSoup(plain, "html.parser")

In [147]:
fields_ibyte = {'Preço': '',
        'Sistema operacional': '',
        'Memória RAM': '',
        'Memória Interna': '',
        'Modelo': '',
        'Cor': '',  
        'Tamanho do Display':'',
        'Avaliações de clientes': '',
        'Marca' : ''
       }

def np_to_dict(table, fields_dict):
    new_dict = fields_dict.copy()
    for cols in table:
        if len(cols) < 3:
            continue
        for key in fields_dict.keys():
            if nomr_l(key, cols[1]) <= 0.25:
                new_dict[key] = cols[2].replace("\n",'')
            #print(nomr_l(key, cols[0]), key, cols[0])
    return new_dict

In [148]:
tb = html_to_np(s)
np_to_dict(tb, fields_ibyte)

{'Preço': '',
 'Sistema operacional': 'Android',
 'Memória RAM': '4GB',
 'Memória Interna': '64GB',
 'Modelo': 'SM-J810M/64DL',
 'Cor': 'Preto',
 'Tamanho do Display': '6"',
 'Avaliações de clientes': '',
 'Marca': 'Samsung'}

# Kabum

In [196]:
url = r"https://www.kabum.com.br/produto/101342/smartphone-samsung-galaxy-a10-32gb-13mp-tela-6-2-preto-sm-a105m-32dl"
code        = requests.get(url, headers=agent)
plain       = code.text
s           = BeautifulSoup(plain, "html.parser")

In [351]:
fields_kabum = {'Preço': '',
        'Sistema operacional': '',
        'Memória RAM': '',
        'Memória Interna': '',
        'Modelo': '',
        'Cor': '',  
        'Tamanho do Display':'',
        'Avaliações de clientes': '',
        'Marca' : ''
       }

def np_to_dict(table, fields_dict):
    new_dict = fields_dict.copy()
    for cols in table:
        if len(cols) < 2:
            continue
        for key in fields_dict.keys():
            if nomr_l(key, cols[0]) <= 0.25:
                new_dict[key] = cols[1].replace("\n",'')
            #print(nomr_l(key, cols[0]), key, cols[0])
    return new_dict

In [352]:
price_roi = s.find_all("div", attrs={"class":"preco_normal"},text=True)
if len(price_roi):
    price = re.findall("R\$ \d{1,5}\,\d{1,3}", str(price_roi[0]))
    if len(price): 
        fields_kabum["Preço"] = price[0]

In [414]:
from lxml import etree
import re
parser = etree.HTMLParser()

XPATH_MODEL = r"""//*[@id="pag-detalhes"]/div/div[6]/div[2]/p/text()"""

tree = etree.parse('kabum.html', parser)

RAW_MODEL = tree.xpath(XPATH_MODEL)

In [426]:
for p in RAW_MODEL:
    if(len(p)):
        for key in fields_kabum:
            if key in p:
                fields_kabum[key] = (p.split(':')[1])
fields_kabum

{'Preço': 'R$ 749,00',
 'Sistema operacional': '',
 'Memória RAM': ' 2GB',
 'Memória Interna': ' 32GB*',
 'Modelo': ' SM-A105M/32DL',
 'Cor': ' 16 Milhões',
 'Tamanho do Display': ' 6.2"*',
 'Avaliações de clientes': '',
 'Marca': ' Samsung'}

# Avenida

In [475]:
fields_avenida = {'Preço': '',
        'Sistema operacional': '',
        'Memória RAM': '',
        'Memória Interna': '',
        'Modelo': '',
        'Cor': '',  
        'Tamanho do Display':'',
        'Avaliações de clientes': '',
        'Marca' : ''
       }

parser = etree.HTMLParser()
XPATH_MODEL = r"""//*[@id="caracteristicas"]/table/tbody/tr[3]"""
tree = etree.parse('avenida.html', parser)

RAW_MODEL = tree.xpath(XPATH_MODEL)

In [476]:
for p in RAW_MODEL:
    print(p)
for p in RAW_MODEL:
    if(len(p)):
        for key in fields_avenida:
            if key in p:
                fields_avenida[key] = (p.split(':')[1])
#fields_avenida

## Cissamagazine

In [521]:
fields_cissamagazine = {'Preço': '',
        'Sistema operacional': '',
        'Memória RAM': '',
        'Memória Interna': '',
        'Modelo': '',
        'Cor': '',  
        'Tamanho do Display':'',
        'Avaliações de clientes': '',
        'Marca' : ''
       }


parser = etree.HTMLParser()
XPATH_DD = r"""//*[@id="caracteristicas"]/div[2]/div/dl/span/dd/text()"""
XPATH_DT = r"""//*[@id="caracteristicas"]/div[2]/div/dl/span/dt/text()"""
             
tree = etree.parse('./cissamagazine.html', parser)

RAW_DD = tree.xpath(XPATH_DD)
RAW_DT = tree.xpath(XPATH_DT)


In [527]:
RAW_DT[1].encode('utf-8').decode()

'VersÃ£o'

In [522]:
for p in zip(RAW_DT, RAW_DD):
    print(p)

('Plataforma', 'Android')
('VersÃ£o', 'Android 9.0 Pie')
('FPS da GravaÃ§Ã£o', '30 fps')
('Tamanho da Tela', '6.39"')
('ResoluÃ§Ã£o', '2340 x 1080')
('Tipo', 'AMOLED')
('NÃºmero de Cores', '16 milhÃµes de cores')
('MemÃ³ria Interna', '128 GB')
('MemÃ³ria RAM', '6 GB')
('MemÃ³ria ExpansÃ\xadvel', '256 GB')
('CÃ¢mera', 'Tripla (48MP + 8MP + 2MP)')
('Recursos', 'Autofocus; ')
('CÃ¢mera Frontal', ' DetecÃ§Ã£o Facial; ')
('Recursos', ' DetecÃ§Ã£o de Sorriso; ')
('Rede', ' EstabilizaÃ§Ã£o de Imagem; ')
('Conectividade', ' Flash; ')
('ProteÃ§Ã£o', ' HDR; ')
('Capacidade', ' Panorama; ')
('RemovÃ\xadvel', ' Temporizador; ')
('Tipo', ' Abertura F/1.8; ')
('Sensores', ' Modo profissional; ')
('Processador', ' Modo Burst')
('Velocidade (Clock)', '32 MP')
('Modelo', 'HDR; ')
('GPU', ' DetecÃ§Ã£o Facial; ')
('Chips', ' Panorama; ')
('Tipo do Chip', ' Temporizador; ')
('VersÃ£o', ' Abertura F/2.0')
('Altura (cm)', '4G+; ')
('Espessura (cm)', ' 4G; ')
('Largura (cm)', ' 3G; ')
('Peso (g)', ' 2G')
('M